<a href="https://colab.research.google.com/github/mjgpinheiro/BitMEX-simple-trading-robot/blob/master/Bitmex_bot2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib
import talib
!pip install bitmex
import bitmex
import numpy as np
import time

# Set up BitMEX client
client = bitmex.bitmex(api_key='<your_api_key>', api_secret='<your_api_secret>')

# Set parameters
symbol = 'XBTUSD' # Trading symbol
order_quantity = 100 # Order quantity
stop_loss = 500 # Stop loss in USD
take_profit = 1000 # Take profit in USD
interval = 60 # Time interval in seconds for checking the price

# Set indicators
window = 14 # Window for calculating RSI and stochastic
k = 3 # Smoothing factor for stochastic

# Loop to check the price
while True:
    # Get historical data
    historical_data = client.Trade.Trade_getBucketed(symbol=symbol, binSize='1h', count=window, reverse=True).result()[0]
    close_price = np.array([data['close'] for data in historical_data])
    high_price = np.array([data['high'] for data in historical_data])
    low_price = np.array([data['low'] for data in historical_data])
    
    # Calculate indicators
    rsi = talib.RSI(close_price, timeperiod=window)[-1]
    slowk, slowd = talib.STOCH(high_price, low_price, close_price, fastk_period=window, slowk_period=k, slowk_matype=0, slowd_period=3, slowd_matype=0)
    stochastic = slowk[-1]
    
    # Check for filters
    if rsi < 30 and stochastic < 20:
        # Place buy order
        client.Order.Order_new(symbol=symbol, orderQty=order_quantity, ordType='Market').result()
    elif rsi > 70 and stochastic > 80:
        # Place sell order
        client.Order.Order_new(symbol=symbol, orderQty=-order_quantity, ordType='Market').result()
    else:
        # Check for risk management
        current_position = client.Position.Position_get(filter={'symbol': symbol}).result()[0][0]
        if current_position['currentQty'] > 0:
            # Check for stop loss
            if current_position['avgEntryPrice'] - current_price <= -stop_loss:
                client.Order.Order_new(symbol=symbol, orderQty=-current_position['currentQty'], ordType='Market').result()
            # Check for take profit
            elif current_price - current_position['avgEntryPrice'] >= take_profit:
                client.Order.Order_new(symbol=symbol, orderQty=-current_position['currentQty'], ordType='Market').result()
        elif current_position['currentQty'] < 0:
            # Check for stop loss
            if current_price - current_position['avgEntryPrice'] >= stop_loss:
                client.Order.Order_new(symbol=symbol, orderQty=-current_position['currentQty'], ordType='Market').result()
            # Check for take profit
            elif current_position['avgEntryPrice'] - current_price >= take_profit:
                client.Order.Order_new(symbol=symbol, orderQty=-current_position['currentQty'], ordType='Market').result()
    
    # Wait for the next interval
    time.sleep(interval)
